In [144]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import abc
from bs4 import BeautifulSoup as bs
import datetime
import math
import re
from requests import get as urlopen


fin = open("DB_original/epi_r.csv", "r", encoding="utf8")
names = []
text = fin.readlines()
for line in text[1:]:
    if '"' in line:
        tmp = '"'.join(line.split('"')[1:-1])
        name = tmp.replace('""', '"')
    else:
        name = line.split(",")[0]
    names += [name.rstrip()]
fin.close()

In [147]:
def handle(spage, name):
    pages = math.ceil(int(spage.find('span',  {'class': 'matching-count'}).text) / 18)
    s1 = " ".join(re.findall("[a-zA-Z]+", name))
    
    for numpage in range(1, pages+1):
        page = bs(urlopen("https://www.epicurious.com/search/" + '%20'.join(name.split()) + \
                  "?page=" + str(numpage)).text, 'lxml')
        cards = [i for i in \
                    page.find_all('article', {'class': 'recipe-content-card'})]
        for card in cards:
            for i in card.find_all('a'):
                s2 = " ".join(re.findall("[a-zA-Z]+", str(i.get('title'))))
                if s1 == s2:
                    return "https://www.epicurious.com/" + i.get('href')

In [149]:
fout = open("DB/bdURL.csv", "w", encoding="utf8")
fout.write(text[0])
for line in range(1, len(text)):
    if line % 500 == 0:
        print(line)
    name = names[line - 1]
    url = handle(bs(urlopen("https://www.epicurious.com/search/" + '%20'.join(name.split())).text, 'lxml'), name)
    if url is None:
        print("FAIL: "+name)
        url="None"
    fout.write(text[line].rstrip()+", \"" + url + "\"\n")
    
    

https://www.epicurious.com//recipes/food/views/coconut-key-lime-sheet-cake
